# Ad Click Prediction

In this notebook, we will be analyzing ad click data from [Kaggle](https://www.kaggle.com/datasets/marius2303/ad-click-prediction-dataset) and building a prediction model.

***

# Initialization

Importing libraries, data etc...

In [1]:
# Setting PYTHONHASHSEED
import os

pyhashseed1 = os.environ.get('PYTHONHASHSEED')
os.environ['PYTHONHASHSEED'] = '0'
pyhashseed2 = os.environ.get('PYTHONHASHSEED')

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    print('Make sure the following says \'None\': ', pyhashseed1)
    print('Make sure the following says \'0\': ', pyhashseed2)

Make sure the following says 'None':  None
Make sure the following says '0':  0


In [3]:
# Importing libraries
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed
np.random.seed(42)

In [10]:
# Import data
adclicks = pd.read_csv("data/ad_click_dataset.csv")

***

# EDA (Exploratory Data Analysis)

This section will be a comprehensive analysis and cleaning of the data. We will examine its structure, remove redundant data, and examine relationships.

## Data Overview

Structure of the dataset, % of missing/duplicate values etc..

In [11]:
# Grab a quick snapshot
adclicks.head()

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,NaN,Desktop,Top,Shopping,Afternoon,1
1,3044,User3044,NaN,Male,Desktop,Top,NaN,NaN,1
2,5912,User5912,41.0,Non-Binary,NaN,Side,Education,Night,1
3,5418,User5418,34.0,Male,NaN,NaN,Entertainment,Evening,1
4,9452,User9452,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0


In [12]:
# Evaluate duplicate values
duplicates = adclicks.duplicated()
print("There are", len(adclicks[duplicates]), "duplicate rows.")

# Drop duplicates
adclicks = adclicks.drop_duplicates()
print("The duplicate rows have been dropped. There are", len(adclicks), "rows remaining.")

There are 2853 duplicate rows.
The duplicate rows have been dropped. There are 7147 rows remaining.


In [14]:
# General information
adclicks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7147 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                7147 non-null   int64  
 1   full_name         7147 non-null   object 
 2   age               3671 non-null   float64
 3   gender            3747 non-null   object 
 4   device_type       5419 non-null   object 
 5   ad_position       5404 non-null   object 
 6   browsing_history  3667 non-null   object 
 7   time_of_day       5435 non-null   object 
 8   click             7147 non-null   int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 558.4+ KB


## Data Cleaning + Data Visualization

Evaluate missing values and think about how to deal with them. Visualize distributions of features and conduct initial visual analysis.